In [ ]:
%cd "drive/MyDrive/cosine_similarity_computation/"
!ls -al

/content/drive/MyDrive/cosine_similarity_computation
total 624530
-rw------- 1 root root 125947272 Oct 31 03:06 __installer__.sh
-rw------- 1 root root  58468498 Jun  7  2018 Miniconda3-4.5.4-Linux-x86_64.sh
drwx------ 2 root root      4096 Oct 15 12:29 rapidsai-csp-utils
-rw------- 1 root root  18274041 Oct 31 02:12 reg_test_15KU.csv
-rw------- 1 root root 221531223 Oct 25 07:22 reg_train_30KU.csv
-rw------- 1 root root    171892 Oct 31 02:55 sample_test_sparse_matrix_15KU_1.5KM.npz
-rw------- 1 root root   2130932 Oct 19 10:28 sample_train_sparse_matrix_28KU_3.5KM.npz
drwx------ 2 root root      4096 Oct 31 03:06 surprise
-rw------- 1 root root  45559912 Mar 21  2018 test_sparse_matrix.npz
-rw------- 1 root root 167424989 Mar 21  2018 train_sparse_matrix.npz


In [ ]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py
!pip3 install cupy-cuda111
!pip3 install scikit-surprise

***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla V100-SXM2-16GB!
***********************************************************************



In [ ]:
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:8 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu bionic InRelease [20.8 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu 

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:24
🔁 Restarting kernel...


In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
%cd "drive/MyDrive/cosine_similarity_computation/"

!python3 rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

In [ ]:
import cupy as cp
import pandas as pd
import numpy as np
from cupyx.scipy.sparse import csr_matrix, find
from sklearn.metrics.pairwise import cosine_similarity
import scipy
import xgboost as xgb
import cuml
import cudf
import rmm
from cuml.metrics.pairwise_distances import pairwise_distances, sparse_pairwise_distances

from datetime import datetime
import os
import random

cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

In [ ]:
train_sparse = csr_matrix(scipy.sparse.load_npz('train_sparse_matrix.npz'), dtype=cp.float32)
test_sparse = csr_matrix(scipy.sparse.load_npz('test_sparse_matrix.npz'), dtype=cp.float32)

In [ ]:
def get_sample_sparse_matrix(sparse_matrix, no_users, no_movies, path, verbose = True):
    """
        It will get it from the ''path'' if it is present  or It will create 
        and store the sampled sparse matrix in the path specified.
    """

    # get (row, col) and (rating) tuple from sparse_matrix...
    row_ind, col_ind, ratings = find(sparse_matrix)
    users = cp.unique(row_ind)
    movies = cp.unique(col_ind)

    print("Original Matrix : (users, movies) -- ({} {})".format(len(users), len(movies)))
    print("Original Matrix : Ratings -- {}\n".format(len(ratings)))

    # It just to make sure to get same sample everytime we run this program..
    # and pick without replacement....
    np.random.seed(15)
    sample_users = cp.random.choice(users, no_users, replace=False)
    sample_movies = cp.random.choice(movies, no_movies, replace=False)
    # get the boolean mask or these sampled_items in original row/col_inds..
    mask = cp.logical_and( cp.isin(row_ind, sample_users),
                      cp.isin(col_ind, sample_movies) )
    
    sample_sparse_matrix = csr_matrix((ratings[mask], (row_ind[mask], col_ind[mask])),
                                             shape=(max(sample_users)+1, max(sample_movies)+1))

    if verbose:
        print("Sampled Matrix : (users, movies) -- ({} {})".format(len(sample_users), len(sample_movies)))
        print("Sampled Matrix : Ratings --", format(ratings[mask].shape[0]))

    print('Saving it into disk for furthur usage..')
    # save it into disk
    scipy.sparse.save_npz(path, sample_sparse_matrix.get())
    if verbose:
            print('Done..\n')
    
    return sample_sparse_matrix

In [ ]:
start = datetime.now()
path = "sample_train_sparse_matrix_30KU_5KM.npz"
if os.path.isfile(path):
    print("It is present in your pwd, getting it from disk....")
    # just get it from the disk instead of computing it
    sample_train_sparse_matrix = csr_matrix(scipy.sparse.load_npz(path), dtype=cp.float32)
    print("DONE..")
else: 
    # get 10k users and 1k movies from available data 
    sample_train_sparse_matrix = get_sample_sparse_matrix(train_sparse, no_users=30000, no_movies=5000,
                                             path = path)

print(datetime.now() - start)

It is present in your pwd, getting it from disk....
DONE..
0:00:00.704481


In [ ]:
start = datetime.now()

path = "sample_test_sparse_matrix_15KU_1.5KM.npz"
if os.path.isfile(path):
    print("It is present in your pwd, getting it from disk....")
    # just get it from the disk instead of computing it
    sample_test_sparse_matrix = csr_matrix(scipy.sparse.load_npz(path), dtype=cp.float32)
    print("DONE..")
else:
    # get 5k users and 500 movies from available data 
    sample_test_sparse_matrix = get_sample_sparse_matrix(test_sparse, no_users=15000, no_movies=1500,
                                                 path=path)
print(datetime.now() - start)

It is present in your pwd, getting it from disk....
DONE..
0:00:00.408112


In [ ]:
# get users, movies and ratings from our samples train sparse matrix
sample_train_users, sample_train_movies, sample_train_ratings = find(sample_train_sparse_matrix)
sample_test_users, sample_test_movies, sample_test_ratings = find(sample_test_sparse_matrix)

type(sample_train_users), len(sample_test_users)

(cupy._core.core.ndarray, 73618)

In [ ]:
sample_train_averages = dict()
# get the global average of ratings in our train set.
global_average = sample_train_sparse_matrix.sum()/sample_train_sparse_matrix.count_nonzero()
sample_train_averages['global'] = global_average

# get the user averages in dictionary (key: user_id/movie_id, value: avg rating)

def get_average_ratings(sparse_matrix, of_users):
    
    # average ratings of user/axes
    ax = 1 if of_users else 0 # 1 - User axes,0 - Movie axes

    # ".A1" is for converting Column_Matrix to 1-D numpy array 
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    # Boolean matrix of ratings ( whether a user rated that movie or not)
    is_rated = sparse_matrix!=0
    # no of ratings that each user OR movie..
    no_of_ratings = is_rated.sum(axis=ax).A1
    
    # max_user  and max_movie ids in sparse matrix 
    u,m = sparse_matrix.shape
    # creae a dictonary of users and their average ratigns..
    average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i]
                                 for i in range(u if of_users else m) 
                                    if no_of_ratings[i] !=0}

    # return that dictionary of average ratings
    return average_ratings

sample_train_averages['user'] = get_average_ratings(sample_train_sparse_matrix.get(), of_users=True)
sample_train_averages['movie'] = get_average_ratings(sample_train_sparse_matrix.get(), of_users=False)

In [ ]:
############################################################
# It took me almost 10 hours to prepare this train dataset.#
############################################################
start = datetime.now()
# if os.path.isfile('reg_train_large.csv'):
#     print("File already exists you don't have to prepare again..." )
    # print('preparing {} tuples for the dataset..\n'.format(len(train_sparse)))
with open('reg_train_large.csv', mode='a') as reg_data_file:
  count = 920000
  index = 920000
  for _, (user, movie, rating)  in enumerate(zip(sample_train_users[920000:], sample_train_movies[920000:], sample_train_ratings[920000:])):
        st = datetime.now()
        user, movie, rating = int(user), int(movie), int(rating)
        #--------------------- Ratings of "movie" by similar users of "user" ---------------------
        # compute the similar Users of the "user"        
        # user_sim = cosine_similarity(sample_train_sparse_matrix[user], sample_train_sparse_matrix).ravel()
        user_sim = 1 - pairwise_distances(sample_train_sparse_matrix[user], sample_train_sparse_matrix, metric="cosine").ravel()
        top_sim_users = user_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
        # get the ratings of most similar users for this movie
        top_ratings = sample_train_sparse_matrix[top_sim_users, movie].toarray().ravel()
        # we will make it's length "5" by adding movie averages to .
        top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
        top_sim_users_ratings.extend([sample_train_averages['movie'][movie]]*(5 - len(top_sim_users_ratings)))
    #     print(top_sim_users_ratings, end=" ")    

        #--------------------- Ratings by "user"  to similar movies of "movie" ---------------------
        # compute the similar movies of the "movie"        
        # movie_sim = cosine_similarity(sample_train_sparse_matrix[:,movie].T, sample_train_sparse_matrix.T).ravel()
        movie_sim = 1 - pairwise_distances(sample_train_sparse_matrix[:, movie].T, sample_train_sparse_matrix.T, metric="cosine").ravel()
        top_sim_movies = movie_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
        # get the ratings of most similar movie rated by this user..
        top_ratings = sample_train_sparse_matrix[user, top_sim_movies].toarray().ravel()
        # we will make it's length "5" by adding user averages to.
        top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
        top_sim_movies_ratings.extend([sample_train_averages['user'][user]]*(5-len(top_sim_movies_ratings))) 
    #     print(top_sim_movies_ratings, end=" : -- ")

        #-----------------prepare the row to be stores in a file-----------------#
        row = list()
        row.append(user)
        row.append(movie)
        # Now add the other features to this data...
        row.append(sample_train_averages['global']) # first feature
        # next 5 features are similar_users "movie" ratings
        row.extend(top_sim_users_ratings)
        # next 5 features are "user" ratings for similar_movies
        row.extend(top_sim_movies_ratings)
        # Avg_user rating
        row.append(sample_train_averages['user'][user])
        # Avg_movie rating
        row.append(sample_train_averages['movie'][movie])

        # finalley, The actual Rating of this user-movie pair...
        row.append(rating)
        count = count + 1
        index = index + 1

        # add rows to the file opened..
        reg_data_file.write(','.join(map(str, row)))
        reg_data_file.write('\n')        
        if (count)%10000 == 0:
            print(index, user, movie)
            # print(','.join(map(str, row)))
            print("Done for {} rows----- {}".format(count, datetime.now() - start))


print(datetime.now() - start)

930000 2021244 13763
Done for 930000 rows----- 0:26:29.036471
940000 2046268 13418
Done for 940000 rows----- 0:53:09.030478
950000 2067192 6339
Done for 950000 rows----- 1:19:26.398436
960000 2093381 7032
Done for 960000 rows----- 1:46:02.842839
970000 2121294 1607
Done for 970000 rows----- 2:12:13.299152
980000 2146325 9946
Done for 980000 rows----- 2:38:29.000041
990000 2175485 14656
Done for 990000 rows----- 3:04:30.853561
1000000 2203332 11478
Done for 1000000 rows----- 3:30:14.785182
1010000 2232611 6235
Done for 1010000 rows----- 3:56:09.441879
1020000 2258079 16495
Done for 1020000 rows----- 4:23:43.152927
1030000 2284783 17506
Done for 1030000 rows----- 4:49:30.983829
1040000 2312320 4216
Done for 1040000 rows----- 5:15:34.208930
1050000 2337846 2072
Done for 1050000 rows----- 5:41:43.429102
1060000 2360593 2391
Done for 1060000 rows----- 6:07:24.093991
1070000 2392643 12125
Done for 1070000 rows----- 6:33:08.431001
1080000 2418149 257
Done for 1080000 rows----- 6:59:39.913742


In [ ]:
start = datetime.now()

if os.path.isfile('reg_test.csv'):
    print("It is already created...")
else:

    print('preparing {} tuples for the dataset..\n'.format(len(sample_test_ratings)))
    with open('reg_test.csv', mode='w') as reg_data_file:
        count = 0 
        for (user, movie, rating)  in zip(sample_test_users, sample_test_movies, sample_test_ratings):
            st = datetime.now()
            user, movie, rating = int(user), int(movie), int(rating)
        #--------------------- Ratings of "movie" by similar users of "user" ---------------------
            #print(user, movie)
            try:
                # compute the similar Users of the "user"        
                user_sim = cosine_similarity(sample_train_sparse_matrix[user].get(), sample_train_sparse_matrix.get()).ravel()
                top_sim_users = user_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
                # get the ratings of most similar users for this movie
                top_ratings = sample_train_sparse_matrix[top_sim_users, movie].toarray().ravel()
                # we will make it's length "5" by adding movie averages to .
                top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
                top_sim_users_ratings.extend([sample_train_averages['movie'][movie]]*(5 - len(top_sim_users_ratings)))
                # print(top_sim_users_ratings, end="--")

            except (IndexError, KeyError):
                # It is a new User or new Movie or there are no ratings for given user for top similar movies...
                ########## Cold STart Problem ##########
                top_sim_users_ratings.extend([sample_train_averages['global']]*(5 - len(top_sim_users_ratings)))
                #print(top_sim_users_ratings)
            except:
                print(user, movie)
                # we just want KeyErrors to be resolved. Not every Exception...
                raise



            #--------------------- Ratings by "user"  to similar movies of "movie" ---------------------
            try:
                # compute the similar movies of the "movie"        
                movie_sim = cosine_similarity(sample_train_sparse_matrix[:,movie].T.get(), sample_train_sparse_matrix.T.get()).ravel()
                top_sim_movies = movie_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
                # get the ratings of most similar movie rated by this user..
                top_ratings = sample_train_sparse_matrix[user, top_sim_movies].get().toarray().ravel()
                # we will make it's length "5" by adding user averages to.
                top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
                top_sim_movies_ratings.extend([sample_train_averages['user'][user]]*(5-len(top_sim_movies_ratings))) 
                #print(top_sim_movies_ratings)
            except (IndexError, KeyError):
                #print(top_sim_movies_ratings, end=" : -- ")
                top_sim_movies_ratings.extend([sample_train_averages['global']]*(5-len(top_sim_movies_ratings)))
                #print(top_sim_movies_ratings)
            except :
                raise

            #-----------------prepare the row to be stores in a file-----------------#
            row = list()
            # add usser and movie name first
            row.append(user)
            row.append(movie)
            row.append(sample_train_averages['global']) # first feature
            #print(row)
            # next 5 features are similar_users "movie" ratings
            row.extend(top_sim_users_ratings)
            #print(row)
            # next 5 features are "user" ratings for similar_movies
            row.extend(top_sim_movies_ratings)
            #print(row)
            # Avg_user rating
            try:
                row.append(sample_train_averages['user'][user])
            except KeyError:
                row.append(sample_train_averages['global'])
            except:
                raise
            #print(row)
            # Avg_movie rating
            try:
                row.append(sample_train_averages['movie'][movie])
            except KeyError:
                row.append(sample_train_averages['global'])
            except:
                raise
            #print(row)
            # finalley, The actual Rating of this user-movie pair...
            row.append(rating)
            #print(row)
            count = count + 1

            # add rows to the file opened..
            reg_data_file.write(','.join(map(str, row)))
            #print(','.join(map(str, row)))
            reg_data_file.write('\n')        
            if (count)%1000 == 0:
                #print(','.join(map(str, row)))
                print("Done for {} rows----- {}".format(count, datetime.now() - start))
    print("",datetime.now() - start)  

preparing 77683 tuples for the dataset..

Done for 1000 rows----- 0:03:55.184439
Done for 2000 rows----- 0:07:48.998480
Done for 3000 rows----- 0:11:42.941655
Done for 4000 rows----- 0:15:33.504789
Done for 5000 rows----- 0:19:25.174344
Done for 6000 rows----- 0:23:15.844064
Done for 7000 rows----- 0:27:07.831746
Done for 8000 rows----- 0:30:57.836056
Done for 9000 rows----- 0:34:51.136102
Done for 10000 rows----- 0:38:38.652524
Done for 11000 rows----- 0:42:23.888916
Done for 12000 rows----- 0:46:09.694808
Done for 13000 rows----- 0:49:49.996715
Done for 14000 rows----- 0:53:37.661921
Done for 15000 rows----- 0:57:16.694860
Done for 16000 rows----- 1:01:02.312640
Done for 17000 rows----- 1:04:39.587320
Done for 18000 rows----- 1:08:23.636830
Done for 19000 rows----- 1:12:07.844179
Done for 20000 rows----- 1:15:48.269356
Done for 21000 rows----- 1:19:26.468834
Done for 22000 rows----- 1:23:07.432268
Done for 23000 rows----- 1:26:49.926652
Done for 24000 rows----- 1:30:30.236362
Done fo

In [ ]:
!pip install xgboost scikit-surprise
%cd "/content/drive/MyDrive/cosine_similarity_computation/"

/content/drive/MyDrive/cosine_similarity_computation


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb

import random

seed = 32
random.seed(seed)
np.random.seed(seed)

In [ ]:
# !git clone https://github.com/NicolasHug/surprise.git
# %cd surprise
reg_train = pd.read_csv('reg_train_30KU.csv')
reg_test = pd.read_csv('reg_test_15KU.csv')

In [ ]:
print('# of unique users in train', len(np.unique(reg_train['user'])))
print('# of unique users in test', len(np.unique(reg_test['user'])))

# of unique users in train 31493
# of unique users in test 11012


In [ ]:
print('# of unique movies in train', len(np.unique(reg_train['movie'])))
print('# of unique movies in test', len(np.unique(reg_test['movie'])))

# of unique movies in train 6999
# of unique movies in test 1279


- We can't give raw data (movie, user, rating) to train the model in Surprise library.


- They have a saperate format for TRAIN and TEST data, which will be useful for training the models like SVD, KNNBaseLineOnly....etc..,in Surprise.


- We can form the trainset from a file, or from a Pandas  DataFrame. 
http://surprise.readthedocs.io/en/stable/getting_started.html#load-dom-dataframe-py 

In [ ]:
from surprise import Reader, Dataset

In [ ]:
# It is to specify how to read the dataframe.
# for our dataframe, we don't have to specify anything extra..
reader = Reader(rating_scale=(1,5))

# create the traindata from the dataframe...
train_data = Dataset.load_from_df(reg_train[['user', 'movie', 'rating']], reader)

# build the trainset from traindata.., It is of dataset format from surprise library..
trainset = train_data.build_full_trainset() 

In [ ]:
testset = list(zip(reg_test.user.values, reg_test.movie.values, reg_test.rating.values))

In [ ]:
models_evaluation_train = dict()
models_evaluation_test = dict()

In [ ]:
# to get rmse and mape given actual and predicted ratings..
def get_error_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean([ (y_true[i] - y_pred[i])**2 for i in range(len(y_pred)) ]))
    mape = np.mean(np.abs( (y_true - y_pred)/y_true )) * 100
    return rmse, mape

###################################################################
###################################################################
def run_xgboost(algo,  x_train, y_train, x_test, y_test, verbose=True):
    """
    It will return train_results and test_results
    """
    
    # dictionaries for storing train and test results
    train_results = dict()
    test_results = dict()
    
    
    # fit the model
    print('Training the model..')
    start = datetime.now()
    # algo.fit(x_train, y_train, eval_metric = 'rmse')
    algo.fit(x_train, y_train)
    print('Done. Time taken : {}\n'.format(datetime.now()-start))
    print('Done \n')

    # from the trained model, get the predictions....
    print('Evaluating the model with TRAIN data...')
    start = datetime.now()
    y_train_pred = algo.predict(x_train)
    # get the rmse and mape of train data...
    rmse_train, mape_train = get_error_metrics(y_train.values, y_train_pred)
    
    # store the results in train_results dictionary..
    train_results = {'rmse': rmse_train,
                    'mape' : mape_train,
                    'predictions' : y_train_pred}
    
    #######################################
    # get the test data predictions and compute rmse and mape
    print('Evaluating Test data')
    y_test_pred = algo.predict(x_test) 
    rmse_test, mape_test = get_error_metrics(y_true=y_test.values, y_pred=y_test_pred)
    # store them in our test results dictionary.
    test_results = {'rmse': rmse_test,
                    'mape' : mape_test,
                    'predictions':y_test_pred}
    if verbose:
        print('\nTEST DATA')
        print('-'*30)
        print('RMSE : ', rmse_test)
        print('MAPE : ', mape_test)
        
    # return these train and test results...
    return train_results, test_results
    

In [ ]:
##########################################################
# get  (actual_list , predicted_list) ratings given list 
# of predictions (prediction is a class in Surprise).    
##########################################################
def get_ratings(predictions):
    actual = np.array([pred.r_ui for pred in predictions])
    pred = np.array([pred.est for pred in predictions])
    
    return actual, pred

################################################################
# get ''rmse'' and ''mape'' , given list of prediction objecs 
################################################################
def get_errors(predictions, print_them=False):

    actual, pred = get_ratings(predictions)
    rmse = np.sqrt(np.mean((pred - actual)**2))
    mape = np.mean(np.abs(pred - actual)/actual)

    return rmse, mape*100

In [ ]:
#################################################################################
# It will return predicted ratings, rmse and mape of both train and test data   #
##################################################################################
def run_surprise(algo, trainset, testset, verbose=True): 
    '''
        return train_dict, test_dict
    
        It returns two dictionaries, one for train and the other is for test
        Each of them have 3 key-value pairs, which specify ''rmse'', ''mape'', and ''predicted ratings''.
    '''
    start = datetime.now()
    # dictionaries that stores metrics for train and test..
    train = dict()
    test = dict()
    
    # train the algorithm with the trainset
    st = datetime.now()
    print('Training the model...')
    algo.fit(trainset)
    print('Done. time taken : {} \n'.format(datetime.now()-st))
    
    # ---------------- Evaluating train data--------------------#
    st = datetime.now()
    print('Evaluating the model with train data..')
    # get the train predictions (list of prediction class inside Surprise)
    train_preds = algo.test(trainset.build_testset())
    # get predicted ratings from the train predictions..
    train_actual_ratings, train_pred_ratings = get_ratings(train_preds)
    # get ''rmse'' and ''mape'' from the train predictions.
    train_rmse, train_mape = get_errors(train_preds)
    print('time taken : {}'.format(datetime.now()-st))
    
    if verbose:
        print('-'*15)
        print('Train Data')
        print('-'*15)
        print("RMSE : {}\n\nMAPE : {}\n".format(train_rmse, train_mape))
    
    #store them in the train dictionary
    if verbose:
        print('adding train results in the dictionary..')
    train['rmse'] = train_rmse
    train['mape'] = train_mape
    train['predictions'] = train_pred_ratings
    
    #------------ Evaluating Test data---------------#
    st = datetime.now()
    print('\nEvaluating for test data...')
    # get the predictions( list of prediction classes) of test data
    test_preds = algo.test(testset)
    # get the predicted ratings from the list of predictions
    test_actual_ratings, test_pred_ratings = get_ratings(test_preds)
    # get error metrics from the predicted and actual ratings
    test_rmse, test_mape = get_errors(test_preds)
    print('time taken : {}'.format(datetime.now()-st))
    
    if verbose:
        print('-'*15)
        print('Test Data')
        print('-'*15)
        print("RMSE : {}\n\nMAPE : {}\n".format(test_rmse, test_mape))
    # store them in test dictionary
    if verbose:
        print('storing the test results in test dictionary...')
    test['rmse'] = test_rmse
    test['mape'] = test_mape
    test['predictions'] = test_pred_ratings
    
    print('\n'+'-'*45)
    print('Total time taken to run this algorithm :', datetime.now() - start)
    
    # return two dictionaries train and test
    return train, test

In [ ]:
%matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.kernel_approximation import Nystroem, AdditiveChi2Sampler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

# prepare Train data
x_train = reg_train.drop(['user','movie','rating'], axis=1)
y_train = reg_train['rating']

# Prepare Test data
x_test = reg_test.drop(['user','movie','rating'], axis=1)
y_test = reg_test['rating']

# initialize Our first XGBoost model...
# first_xgb = xgb.XGBRegressor(silent=False, n_jobs=13, random_state=200, n_estimators=15, gpu_id=0, tree_method='gpu_hist', reg_lambda=1, 
#                              sampling_method="gradient_based", max_bin=4096, learning_rate=0.3,
#                              max_depth=15, booster='dart')
# first_xgb = SGDRegressor(loss="huber", learning_rate="adaptive", penalty="l2")
# kernel_transformer = AdditiveChi2Sampler()

# x_train = kernel_transformer.fit_transform(x_train)
# x_test = kernel_transformer.transform(x_test)

first_xgb = SGDRegressor(loss="huber", learning_rate="adaptive", penalty="l2")
train_results, test_results = run_xgboost(first_xgb, x_train, y_train, x_test, y_test)

# store the results in models_evaluations dictionaries
models_evaluation_train['first_algo'] = train_results
models_evaluation_test['first_algo'] = test_results

# xgb.plot_importance(first_xgb)
# plt.show()

Using matplotlib backend: agg
Training the model..
Done. Time taken : 0:00:34.528700

Done 

Evaluating the model with TRAIN data...
Evaluating Test data

TEST DATA
------------------------------
RMSE :  1.0760162324179263
MAPE :  32.763482502383305


__Predicted_rating : ( baseline prediction ) __

    -  http://surprise.readthedocs.io/en/stable/basic_algorithms.html#surprise.prediction_algorithms.baseline_only.BaselineOnly 
 >$   \large {\hat{r}_{ui} = b_{ui} =\mu + b_u + b_i} $


- $\pmb \mu $ : Average of all trainings in training data.
- $\pmb b_u$ : User bias
- $\pmb b_i$ : Item bias (movie biases) 

__Optimization function ( Least Squares Problem ) __

    - http://surprise.readthedocs.io/en/stable/prediction_algorithms.html#baselines-estimates-configuration 

> $ \large \sum_{r_{ui} \in R_{train}} \left(r_{ui} - (\mu + b_u + b_i)\right)^2 +
\lambda \left(b_u^2 + b_i^2 \right).\text {        [mimimize } {b_u, b_i]}$ 

In [ ]:
from surprise import BaselineOnly 

In [ ]:
# options are to specify.., how to compute those user and item biases
bsl_options = {'method': 'sgd',
               'learning_rate': 0.01
               }
bsl_algo = BaselineOnly(bsl_options=bsl_options)
# run this algorithm.., It will return the train and test results..
bsl_train_results, bsl_test_results = run_surprise(bsl_algo, trainset, testset, verbose=True)


# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['bsl_algo'] = bsl_train_results
models_evaluation_test['bsl_algo'] = bsl_test_results

Training the model...
Estimating biases using sgd...
Done. time taken : 0:00:04.838103 

Evaluating the model with train data..
time taken : 0:00:15.033884
---------------
Train Data
---------------
RMSE : 3267.872247633649

MAPE : 46.3947695136529

adding train results in the dictionary..

Evaluating for test data...
time taken : 0:00:00.688662
---------------
Test Data
---------------
RMSE : 1.5165482049240204

MAPE : 47.90767652723992

storing the test results in test dictionary...

---------------------------------------------
Total time taken to run this algorithm : 0:00:20.561378


In [ ]:
# add our baseline_predicted value as our feature..
reg_train['bslpr'] = models_evaluation_train['bsl_algo']['predictions']
reg_train.head(2)

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,bslpr
0,7,30,3.585633,4.0,5.0,5.0,3.0,5.0,3.0,4.0,4.0,5.0,4.0,4.007812,3.790133,5,4.329560
1,7,185,3.585633,4.0,5.0,3.0,4.0,1.0,4.0,4.0,3.0,3.0,3.0,4.007812,2.944099,4,3.630428


In [ ]:
reg_test['bslpr']  = models_evaluation_test['bsl_algo']['predictions']
reg_test.head(2)

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,bslpr
0,8,3936,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,4,5.0
1,8,6386,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,5,5.0


In [ ]:
# prepare train data
x_train = reg_train.drop(['user', 'movie','rating'], axis=1)
y_train = reg_train['rating']

# Prepare Test data
x_test = reg_test.drop(['user','movie','rating'], axis=1)
y_test = reg_test['rating']

# initialize Our first XGBoost model...
# xgb_bsl = xgb.XGBRegressor(silent=False, n_jobs=13, random_state=15, n_estimators=100, gpu_id=0, tree_method='gpu_hist')
xgb_bsl = SGDRegressor(loss="huber", learning_rate="adaptive", penalty="elasticnet")
train_results, test_results = run_xgboost(xgb_bsl, x_train, y_train, x_test, y_test)

# store the results in models_evaluations dictionaries
models_evaluation_train['xgb_bsl'] = train_results
models_evaluation_test['xgb_bsl'] = test_results

# xgb.plot_importance(xgb_bsl)
# plt.show()


Training the model..
Done. Time taken : 0:00:31.812145

Done 

Evaluating the model with TRAIN data...
Evaluating Test data

TEST DATA
------------------------------
RMSE :  1.0767756364568846
MAPE :  32.728655803765854


- KNN BASELINE
    - http://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBaseline 

In [ ]:
from surprise import KNNBaseline, KNNWithZScore

In [ ]:
# we specify , how to compute similarities and what to consider with sim_options to our algorithm
sim_options = {'user_based' : True,
               'name': 'pearson_baseline',
               'shrinkage': 100,
               'min_support': 2
              } 
# we keep other parameters like regularization parameter and learning_rate as default values.
bsl_options = {'method': 'sgd'} 

knn_bsl_u = KNNWithZScore(k=40, sim_options = sim_options, bsl_options = bsl_options)
knn_bsl_u_train_results, knn_bsl_u_test_results = run_surprise(knn_bsl_u, trainset, testset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['knn_bsl_u'] = knn_bsl_u_train_results 
models_evaluation_test['knn_bsl_u'] = knn_bsl_u_test_results


Training the model...
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done. time taken : 0:21:19.868142 

Evaluating the model with train data..
time taken : 2:15:09.648990
---------------
Train Data
---------------
RMSE : 3267.8717881322973

MAPE : 23.11124309680427

adding train results in the dictionary..

Evaluating for test data...
time taken : 0:00:06.310134
---------------
Test Data
---------------
RMSE : 1.6644940383654292

MAPE : 54.62470383960838

storing the test results in test dictionary...

---------------------------------------------
Total time taken to run this algorithm : 2:36:35.829399


- PEARSON_BASELINE SIMILARITY
    - http://surprise.readthedocs.io/en/stable/similarities.html#surprise.similarities.pearson_baseline 

- SHRINKAGE
    - _2.2 Neighborhood Models_ in http://courses.ischool.berkeley.edu/i290-dm/s11/SECURE/a1-koren.pdf 

- __predicted Rating__ : ( ___ based on User-User similarity ___ )

\begin{align} \hat{r}_{ui} = b_{ui} + \frac{ \sum\limits_{v \in N^k_i(u)}
\text{sim}(u, v) \cdot (r_{vi} - b_{vi})} {\sum\limits_{v \in
N^k_i(u)} \text{sim}(u, v)} \end{align}

- $\pmb{b_{ui}}$ -  _Baseline prediction_ of (user,movie) rating

- $ \pmb {N_i^k (u)}$ - Set of __K similar__ users (neighbours) of __user (u)__ who rated __movie(i)__  

- _sim (u, v)_ - __Similarity__ between users __u and v__  
    - Generally, it will be cosine similarity or Pearson correlation coefficient. 
    - But we use __shrunk Pearson-baseline correlation coefficient__, which is based on the pearsonBaseline similarity ( we take base line predictions instead of mean rating of user/item)
       

- __ Predicted rating __ ( based on Item Item similarity ):
 \begin{align} \hat{r}_{ui} = b_{ui} + \frac{ \sum\limits_{j \in N^k_u(i)}\text{sim}(i, j) \cdot (r_{uj} - b_{uj})} {\sum\limits_{j \in N^k_u(j)} \text{sim}(i, j)} \end{align}

    -  ___Notations follows same as above (user user based predicted rating ) ___

In [ ]:
# we specify , how to compute similarities and what to consider with sim_options to our algorithm

# 'user_based' : Fals => this considers the similarities of movies instead of users

sim_options = {'user_based' : False,
               'name': 'pearson_baseline',
               'shrinkage': 100,
               'min_support': 2
              } 
# we keep other parameters like regularization parameter and learning_rate as default values.
bsl_options = {'method': 'sgd'}


knn_bsl_m = KNNWithZScore(k=40, sim_options = sim_options, bsl_options = bsl_options)

knn_bsl_m_train_results, knn_bsl_m_test_results = run_surprise(knn_bsl_m, trainset, testset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['knn_bsl_m'] = knn_bsl_m_train_results 
models_evaluation_test['knn_bsl_m'] = knn_bsl_m_test_results

Training the model...
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Done. time taken : 0:00:30.237522 

Evaluating the model with train data..
time taken : 0:06:20.722268
---------------
Train Data
---------------
RMSE : 3267.87180605701

MAPE : 25.610201860816506

adding train results in the dictionary..

Evaluating for test data...
time taken : 0:00:00.893109
---------------
Test Data
---------------
RMSE : 1.663634521380457

MAPE : 54.588593609838576

storing the test results in test dictionary...

---------------------------------------------
Total time taken to run this algorithm : 0:06:51.855800


In [ ]:
# add the predicted values from both knns to this dataframe
reg_train['knn_bsl_u'] = models_evaluation_train['knn_bsl_u']['predictions']
reg_train['knn_bsl_m'] = models_evaluation_train['knn_bsl_m']['predictions']
reg_train.head(2)

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,bslpr,knn_bsl_u,knn_bsl_m
0,7,30,3.585633,4.0,5.0,5.0,3.0,5.0,3.0,4.0,4.0,5.0,4.0,4.007812,3.790133,5,4.329560,4.301667,4.309889
1,7,185,3.585633,4.0,5.0,3.0,4.0,1.0,4.0,4.0,3.0,3.0,3.0,4.007812,2.944099,4,3.630428,3.773317,3.637829


In [ ]:
reg_test['knn_bsl_u'] = models_evaluation_test['knn_bsl_u']['predictions']
reg_test['knn_bsl_m'] = models_evaluation_test['knn_bsl_m']['predictions']

reg_test.head(2)

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,bslpr,knn_bsl_u,knn_bsl_m
0,8,3936,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,4,5.0,5.0,5.0
1,8,6386,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,5,5.0,5.0,5.0


In [ ]:
# prepare the train data....
x_train = reg_train.drop(['user', 'movie', 'rating'], axis=1)
y_train = reg_train['rating']

# prepare the train data....
x_test = reg_test.drop(['user','movie','rating'], axis=1)
y_test = reg_test['rating']

# declare the model
# xgb_knn_bsl = xgb.XGBRegressor(n_jobs=10, random_state=15, gpu_id=0, tree_method='gpu_hist')
xgb_knn_bsl = SGDRegressor(loss="huber", learning_rate="adaptive", penalty="l2")
train_results, test_results = run_xgboost(xgb_knn_bsl, x_train, y_train, x_test, y_test)

# store the results in models_evaluations dictionaries
models_evaluation_train['xgb_knn_bsl'] = train_results
models_evaluation_test['xgb_knn_bsl'] = test_results


# xgb.plot_importance(xgb_knn_bsl)
# plt.show()

Training the model..
Done. Time taken : 0:00:32.098960

Done 

Evaluating the model with TRAIN data...
Evaluating Test data

TEST DATA
------------------------------
RMSE :  1.0706272280968663
MAPE :  32.925854476335005


http://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD 

- __ Predicted Rating : __
    - 
    - $ \large  \hat r_{ui} = \mu + b_u + b_i + q_i^Tp_u $
    
        - $\pmb q_i$ - Representation of item(movie) in latent factor space
        
        - $\pmb p_u$ - Representation of user in new latent factor space
        


- A BASIC MATRIX FACTORIZATION MODEL in  https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf

- __Optimization problem with user item interactions and regularization (to avoid overfitting)__
    - 
    - $\large \sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
\lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right) $

In [ ]:
from surprise import SVD

In [ ]:
# initiallize the model
svd = SVD(n_factors=100, biased=True, random_state=15, verbose=True)
svd_train_results, svd_test_results = run_surprise(svd, trainset, testset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['svd'] = svd_train_results 
models_evaluation_test['svd'] = svd_test_results

Training the model...
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Done. time taken : 0:01:35.573361 

Evaluating the model with train data..
time taken : 0:00:19.522100
---------------
Train Data
---------------
RMSE : 3267.8721799055943

MAPE : 61.85151835050975

adding train results in the dictionary..

Evaluating for test data...
time taken : 0:00:01.312970
---------------
Test Data
---------------
RMSE : 1.6777399910041546

MAPE : 55.25812168256461

storing the test results in test dictionary...

---------------------------------------------
Total time taken to run this algorithm : 0:01:56.411442


- ----->  2.5 Implicit Feedback in http://courses.ischool.berkeley.edu/i290-dm/s11/SECURE/a1-koren.pdf

- __ Predicted Rating : __
    - $ \large \hat{r}_{ui} = \mu + b_u + b_i + q_i^T\left(p_u +
    |I_u|^{-\frac{1}{2}} \sum_{j \in I_u}y_j\right) $ 

 - $ \pmb{I_u}$ --- the set of all items rated by user u

- $\pmb{y_j}$ --- Our new set of item factors that capture implicit ratings.  

- __Optimization problem with user item interactions and regularization (to avoid overfitting)__
    - 
    - $ \large \sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
\lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2 + ||y_j||^2\right) $ 

In [ ]:
from surprise import SVDpp

In [ ]:
# initiallize the model
svdpp = SVDpp(n_factors=50, random_state=15, verbose=True)
svdpp_train_results, svdpp_test_results = run_surprise(svdpp, trainset, testset, verbose=True)

# Just store these error metrics in our models_evaluation datastructure
models_evaluation_train['svdpp'] = svdpp_train_results 
models_evaluation_test['svdpp'] = svdpp_test_results


Training the model...
 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
Done. time taken : 2:13:06.618435 

Evaluating the model with train data..
time taken : 0:05:30.395810
---------------
Train Data
---------------
RMSE : 3267.8721799055943

MAPE : 61.85151835050975

adding train results in the dictionary..

Evaluating for test data...
time taken : 0:00:01.521087
---------------
Test Data
---------------
RMSE : 1.6777399910041546

MAPE : 55.25812168256461

storing the test results in test dictionary...

---------------------------------------------
Total time taken to run this algorithm : 2:18:38.537986


In [ ]:
# add the predicted values from both knns to this dataframe
reg_train['svd'] = models_evaluation_train['svd']['predictions']
reg_train['svdpp'] = models_evaluation_train['svdpp']['predictions']

reg_train.head(2) 

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,bslpr,knn_bsl_u,knn_bsl_m,svd,svdpp
0,7,30,3.585633,4.0,5.0,5.0,3.0,5.0,3.0,4.0,4.0,5.0,4.0,4.007812,3.790133,5,4.329560,4.301667,4.309889,5,5
1,7,185,3.585633,4.0,5.0,3.0,4.0,1.0,4.0,4.0,3.0,3.0,3.0,4.007812,2.944099,4,3.630428,3.773317,3.637829,5,5


In [ ]:
reg_test['svd'] = models_evaluation_test['svd']['predictions']
reg_test['svdpp'] = models_evaluation_test['svdpp']['predictions']

reg_test.head(2) 

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,bslpr,knn_bsl_u,knn_bsl_m,svd,svdpp
0,8,3936,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,4,5.0,5.0,5.0,5,5
1,8,6386,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,3.578906,5,5.0,5.0,5.0,5,5


In [ ]:
# prepare x_train and y_train
x_train = reg_train.drop(['user', 'movie', 'rating',], axis=1)
y_train = reg_train['rating']

# prepare test data
x_test = reg_test.drop(['user', 'movie', 'rating'], axis=1)
y_test = reg_test['rating']


# xgb_final = xgb.XGBRegressor(n_jobs=10, random_state=15, gpu_id=0)
xgb_final = SGDRegressor(loss="huber", learning_rate="adaptive", penalty="l2")
train_results, test_results = run_xgboost(xgb_final, x_train, y_train, x_test, y_test)

# store the results in models_evaluations dictionaries
models_evaluation_train['xgb_final'] = train_results
models_evaluation_test['xgb_final'] = test_results


# xgb.plot_importance(xgb_final)
# plt.show()

In [ ]:
# prepare train data
x_train = reg_train[['knn_bsl_u', 'knn_bsl_m', 'svd', 'svdpp']]
y_train = reg_train['rating']

# test data
x_test = reg_test_df[['knn_bsl_u', 'knn_bsl_m', 'svd', 'svdpp']]
y_test = reg_test_df['rating']


# xgb_all_models = xgb.XGBRegressor(n_jobs=10, random_state=15, gpu_id=0)
xgb_all_models = SGDRegressor(loss="huber", learning_rate="adaptive", penalty="l2")
train_results, test_results = run_xgboost(xgb_all_models, x_train, y_train, x_test, y_test)

# store the results in models_evaluations dictionaries
models_evaluation_train['xgb_all_models'] = train_results
models_evaluation_test['xgb_all_models'] = test_results

# xgb.plot_importance(xgb_all_models)
# plt.show()

In [ ]:
# Saving our TEST_RESULTS into a dataframe so that you don't have to run it again
pd.DataFrame(models_evaluation_test).to_csv('sample_results.csv')
models = pd.read_csv('sample_results.csv', index_col=0)
models.loc['rmse'].sort_values()